## 主函数

In [1]:
# -*- coding: utf-8 -*-
"""
双流端到端自动驾驶网络 (Dual-Stream End-to-End Driving Model)
- 视觉流: ResNet_CBAM + Transformer (处理 t-2, t-1, t 三帧图像)
- 状态流: LSTM (处理过去 N 帧的 [速度, 加速度, 转角])
- 融合策略: Output = MLP(Concat(Visual, LSTM)) + LSTM
- 预测输出: 当前时刻所需的 [加速度, 转角]
"""

# 还原数据
# target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']


处理完成！归一化参数已保存至 scaler_params.json，序列化数据已保存至 global_vehicle_data_history_cols.csv


## 优化前版本，已经弃用

In [ ]:

import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# 导入你自己写的各个模块
from dataset_create import prepare_dataset_and_scaler, ProcessedDrivingDataset, inverse_transform
from model import DualStreamDrivingModel

def main():
    # 1. 配置文件路径
    input_csv = 'global_vehicle_data.csv'
    output_csv = 'global_vehicle_data_history_cols.csv'
    scaler_json = 'scaler_params.json'
    target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']

    # 2. 如果之前没有预处理数据，则调用预处理（这里可以按需注释）
    print(">>> 开始检查并预处理数据...")
    scaler_params = prepare_dataset_and_scaler(
        input_file=input_csv,
        output_csv=output_csv,
        scaler_json_path=scaler_json,
        seq_length=9
    )
    print(">>> 预处理完成！\n")

    # 3. 加载数据集
    dataset = ProcessedDrivingDataset(csv_file=output_csv)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

    # 4. 初始化端到端模型和损失函数
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DualStreamDrivingModel().to(device)
    criterion = nn.MSELoss()

    print(f">>> 模型初始化成功，运行在 {device} 上，开始测试 Forward Pass...\n")

    # 5. 模拟训练循环
    for batch_idx, (images, state_seq, target) in enumerate(dataloader):
        # 数据移至对应设备
        img_t_minus_2, img_t_minus_1, img_t = [img.to(device) for img in images]
        state_seq = state_seq.to(device)
        target = target.to(device)

        # 模型前向传播
        predictions = model(img_t_minus_2, img_t_minus_1, img_t, state_seq)
        loss = criterion(predictions, target)

        # 打印调试信息
        print(f"--- Batch {batch_idx} ---")
        print(f"图像尺寸: {img_t.shape}")
        print(f"LSTM 输入尺寸: {state_seq.shape} -> (Batch=4, Seq=8, Features=10)")
        print(f"预测输出尺寸: {predictions.shape}")
        print(f"当前 Loss: {loss.item():.4f}")

        # 测试反归一化还原 (取 Batch 中的第 0 个样本)
        pred_numpy = predictions[0].detach().cpu().numpy()
        real_values = inverse_transform(pred_numpy, target_names, scaler_params)
        print(f"\n模型下发的真实控制指令 [acc_x, acc_y, acc_z, steer]: \n{real_values}\n")

        # 只跑一个 Batch 验证
        break

if __name__ == "__main__":
    main()

## 训练代码

In [1]:
import os
import json
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from dataset_create import prepare_dataset_and_scaler, ProcessedDrivingDataset, inverse_transform
from model import DualStreamDrivingModel
from datetime import datetime

def train():
    # 配置文件路径
    input_csv = 'global_vehicle_data.csv'
    output_csv = 'global_vehicle_data_history_cols.csv'
    scaler_json = 'scaler_params.json'
    target_names = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'steer']

    # 数据预处理
    print(">>> 开始检查并预处理数据...")
    scaler_params = prepare_dataset_and_scaler(
        input_file=input_csv,
        output_csv=output_csv,
        scaler_json_path=scaler_json,
        seq_length=9
    )
    print(">>> 预处理完成！\n")

    # 加载数据集
    dataset = ProcessedDrivingDataset(csv_file=output_csv)
    dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=0)

    # 初始化模型、损失函数、优化器
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = DualStreamDrivingModel().to(device)
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=1e-4)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.9)

    # 日志文件
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    log_file = os.path.join(log_dir, f"training_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt")

    # 训练循环
    num_epochs = 20
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0

        for batch_idx, (images, state_seq, target) in enumerate(dataloader):
            # 数据移至设备
            img_t_minus_2, img_t_minus_1, img_t = [img.to(device) for img in images]
            state_seq = state_seq.to(device)
            target = target.to(device)

            # 前向传播
            predictions = model(img_t_minus_2, img_t_minus_1, img_t, state_seq)
            loss = criterion(predictions, target)

            # 反向传播与优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

            # 打印当前 Batch 的损失
            print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(dataloader)}], Loss: {loss.item():.4f}")

        # 学习率调度
        scheduler.step()

        # 记录日志
        with open(log_file, "a") as f:
            f.write(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(dataloader):.4f}\n")

        print(f"Epoch [{epoch+1}/{num_epochs}] 完成，平均 Loss: {epoch_loss/len(dataloader):.4f}")

    # 保存模型
    model_path = os.path.join(log_dir, "dual_stream_model.pth")
    torch.save(model.state_dict(), model_path)
    print(f"模型已保存至 {model_path}")

if __name__ == "__main__":
    train()

>>> 开始检查并预处理数据...
>>> 预处理完成！



UnidentifiedImageError: cannot identify image file 'carla_data_collect/20260130_164858/images/front/237_966510_front.png'